# Notebook eksperimen skripsi

## Import library and dataset

In [1]:
import pandas as pd
import Levenshtein
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer
from fuzzywuzzy import fuzz
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.spatial.distance import cosine


In [2]:
data = pd.read_json('../data/final_data/data_all.jsonl', lines=True)

## Pre-Processing

In [3]:
# Hitung jumlah dan persentase match_type
match_count = data['match_type'].value_counts()
total_count = len(data)

percentage_match = (match_count['Match'] / total_count) * 100 if 'Match' in match_count else 0
percentage_non_match = (match_count['Non-Match'] / total_count) * 100 if 'Non-Match' in match_count else 0

print(f"Jumlah Baris Match: {match_count.get('Match', 0)} ({percentage_match:.2f}%)")
print(f"Jumlah Baris Non-Match: {match_count.get('Non-Match', 0)} ({percentage_non_match:.2f}%)")
print(f"Jumlah Baris: {total_count}")

Jumlah Baris Match: 12125 (52.19%)
Jumlah Baris Non-Match: 11109 (47.81%)
Jumlah Baris: 23234


In [4]:
# Rename & Replace nama dan nilai kolom
data.rename(columns={'match_type': 'match'}, inplace=True)
data['match'] = data['match'].map({'Match': 1, 'Non-Match': 0})
data

,c1,c1_context,c2,c2_context,match
0,anz,The Australia and New Zealand Banking Group Li...,ANZ,We provide banking and financial products and ...,1
1,anz,The Australia and New Zealand Banking Group Li...,Australian New Zealand Banking Group,ANZ\r\n- The Australian New Zealand Banking Gr...,1
2,anz,The Australia and New Zealand Banking Group Li...,A.N.Z.,We provide banking and financial products and ...,1
3,anz,The Australia and New Zealand Banking Group Li...,ANZ Bank,ANZ Bank New Zealand Limited operates as a ban...,1
4,anz,The Australia and New Zealand Banking Group Li...,Anz,We provide banking and financial products and ...,1
...,...,...,...,...,...
23229,aerotek,Aerotek\nprovides staffing and services soluti...,AeroVironment,AeroVironment (NASDAQ: AVAV) is a technology s...,0
23230,aerotek,Aerotek\nprovides staffing and services soluti...,Aeropostale,Aéropostale is a specialty retailer of high-qu...,0
23231,michigan state university,Michigan State University is the nation's prem...,Michigan Technological University,Michigan Technological University is a flagshi...,0
23232,michigan state university,Michigan State University is the nation's prem...,Central Michigan University,Central Michigan University is a leading publi...,0


## Baseline Models (Decission Tree)

### Levenshtein Distance

In [5]:
def calculate_levenshtein(row):
    c1 = row['c1'].lower()
    c2 = row['c2'].lower()
    return Levenshtein.distance(c1, c2)

data['levensthein_distance'] = data.apply(calculate_levenshtein, axis=1)


**Data Split and Model Training**

In [6]:
# Pilih fitur dan target
features = data[['levensthein_distance']] 
target = data['match']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Melatih model Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Melakukan prediksi
y_pred = model.predict(X_test)

# Menampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.72      0.70      2203
           1       0.74      0.71      0.72      2444

    accuracy                           0.71      4647
   macro avg       0.71      0.71      0.71      4647
weighted avg       0.71      0.71      0.71      4647



### N-Gram Similarity (with Jaccard)
Buat n-gram, lalu cari distancenya menggunakan Jaccard

In [7]:
# Fungsi untuk membuat n-gram (Bi-Gram 2)
def ngram_generator(text, n=2):
    """
    text: String yang akan dipecah menjadi n-gram
    n : panjang n-gram yang akan di buat
    """
    return [text[i:i+n] for i in range(len(text) - n + 1)]

# Fungsi untuk menghitung n-gram similarity
def calculate_ngram_similarity(row):
    """
    row: baris pada dataframe (c1,c2)
    n: panjang n-gram
    """

    c1 = row['c1'].lower()
    c2 = row['c2'].lower()

    ngrams_c1 = ngram_generator(c1)
    ngrams_c2 = ngram_generator(c2)

    # Menggunakan MultiLabelBinarizer untuk menghitung Jaccard similarity
    mlb = MultiLabelBinarizer()
    # ngrams = list(set(ngrams_c1) | set(ngrams_c2))  # Semua n-gram unik
    binarized = mlb.fit_transform([ngrams_c1, ngrams_c2])
    
    return jaccard_score(binarized[0], binarized[1])

data['ngram_similarity'] = data.apply(calculate_ngram_similarity, axis=1)

**Data Split and Model Training**

In [8]:
# Pilih fitur dan target
features = data[['ngram_similarity']] 
target = data['match']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Melatih model Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Melakukan prediksi
y_pred = model.predict(X_test)

# Menampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.82      0.79      2203
           1       0.82      0.77      0.80      2444

    accuracy                           0.80      4647
   macro avg       0.80      0.80      0.80      4647
weighted avg       0.80      0.80      0.80      4647



### Token Sort Ratio

Token sort ratio tidak peduli urutan kata dalam suatu kalimat, Cocok untuk company matching

Contoh:
Bank Mandiri - Mandiri Bank

Bank Central Asia - Asia Central Bank

Bayer AG - AG Bayer

Nestlé S.A. - S.A. Nestlé

In [9]:
# Check the similarity score with different names
full_name_1 = "Alice B. Johnson"
full_name_2 = "Johnson Alice B."

# Order does not matter for token sort ratio
print(f"Token sort ratio similarity score: {fuzz.token_sort_ratio(full_name_2, full_name_1)}")

# Order matters for partial ratio
print(f"Partial ratio similarity score: {fuzz.partial_ratio(full_name_1, full_name_2)}")

# Order will not affect simple ratio if strings do not match
print(f"Simple ratio similarity score: {fuzz.ratio(full_name_1, full_name_2)}")


Token sort ratio similarity score: 100
Partial ratio similarity score: 67
Simple ratio similarity score: 50


In [10]:
# Fungsi untuk menghitung token sort ratio
def calculate_token_partial_ratio(row):
    c1 = row['c1'].lower()
    c2 = row['c2'].lower()
    return fuzz.partial_ratio(c1, c2)

data['token_partial_ratio'] = data.apply(calculate_token_partial_ratio, axis=1)

**Data Split and Model Training**

In [11]:
# Pilih fitur dan target
features = data[['token_partial_ratio']] 
target = data['match']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Melatih model Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Melakukan prediksi
y_pred = model.predict(X_test)

# Menampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84      2203
           1       0.87      0.82      0.84      2444

    accuracy                           0.84      4647
   macro avg       0.84      0.84      0.84      4647
weighted avg       0.84      0.84      0.84      4647



# Dataframe final

In [12]:
selected_data = data[["c1","c2","levensthein_distance","ngram_similarity","token_partial_ratio","match"]]
selected_data

,c1,c2,levensthein_distance,ngram_similarity,token_partial_ratio,match
0,anz,ANZ,0,1.000000,100,1
1,anz,Australian New Zealand Banking Group,33,0.030303,67,1
2,anz,A.N.Z.,3,0.000000,67,1
3,anz,ANZ Bank,5,0.333333,100,1
4,anz,Anz,0,1.000000,100,1
...,...,...,...,...,...,...
23229,aerotek,AeroVironment,8,0.214286,57,0
23230,aerotek,Aeropostale,6,0.230769,57,0
23231,michigan state university,Michigan Technological University,13,0.542857,61,0
23232,michigan state university,Central Michigan University,14,0.562500,86,0


## Ensemble Learning

Menambahkan fitur baru

In [13]:
# Menghitung jumlah kata dan jumlah karakter
data['number_of_words_c1'] = data['c1'].apply(lambda x: len(x.split()))
data['number_of_characters_c1'] = data['c1'].apply(lambda x: len(x))

data['number_of_words_c2'] = data['c2'].apply(lambda x: len(x.split()))
data['number_of_characters_c2'] = data['c2'].apply(lambda x: len(x))

data

,c1,c1_context,c2,c2_context,match,levensthein_distance,ngram_similarity,token_partial_ratio,number_of_words_c1,number_of_characters_c1,number_of_words_c2,number_of_characters_c2
0,anz,The Australia and New Zealand Banking Group Li...,ANZ,We provide banking and financial products and ...,1,0,1.000000,100,1,3,1,3
1,anz,The Australia and New Zealand Banking Group Li...,Australian New Zealand Banking Group,ANZ\r\n- The Australian New Zealand Banking Gr...,1,33,0.030303,67,1,3,5,36
2,anz,The Australia and New Zealand Banking Group Li...,A.N.Z.,We provide banking and financial products and ...,1,3,0.000000,67,1,3,1,6
3,anz,The Australia and New Zealand Banking Group Li...,ANZ Bank,ANZ Bank New Zealand Limited operates as a ban...,1,5,0.333333,100,1,3,2,8
4,anz,The Australia and New Zealand Banking Group Li...,Anz,We provide banking and financial products and ...,1,0,1.000000,100,1,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
23229,aerotek,Aerotek\nprovides staffing and services soluti...,AeroVironment,AeroVironment (NASDAQ: AVAV) is a technology s...,0,8,0.214286,57,1,7,1,13
23230,aerotek,Aerotek\nprovides staffing and services soluti...,Aeropostale,Aéropostale is a specialty retailer of high-qu...,0,6,0.230769,57,1,7,1,11
23231,michigan state university,Michigan State University is the nation's prem...,Michigan Technological University,Michigan Technological University is a flagshi...,0,13,0.542857,61,3,25,3,33
23232,michigan state university,Michigan State University is the nation's prem...,Central Michigan University,Central Michigan University is a leading publi...,0,14,0.562500,86,3,25,3,27


### XGBoost

Combine all feature

In [14]:
# Pilih fitur dan target
features = data[['levensthein_distance','ngram_similarity','token_partial_ratio']] 
target = data['match']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Melatih model Decision Tree
model = XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

# Melakukan prediksi
y_pred = model.predict(X_test)

# Menampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85      2203
           1       0.86      0.87      0.86      2444

    accuracy                           0.85      4647
   macro avg       0.85      0.85      0.85      4647
weighted avg       0.85      0.85      0.85      4647



In [15]:
# Pilih fitur dan target
features = data[['levensthein_distance','ngram_similarity','token_partial_ratio', 'number_of_words_c1', 'number_of_words_c2', 'number_of_characters_c1', 'number_of_characters_c2']] 
target = data['match']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Melatih model Decision Tree
model = XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

# Melakukan prediksi
y_pred = model.predict(X_test)

# Menampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2203
           1       0.89      0.90      0.90      2444

    accuracy                           0.89      4647
   macro avg       0.89      0.89      0.89      4647
weighted avg       0.89      0.89      0.89      4647



## Deep Learning

1. Pre-processing context (tanda baca, dll)
2. Tokenization (perlukah ?)
3. Vectorization
4. Training BERT model
5. Testing/Evaluation 

### Tanpa Konteks

#### Buat Fitur

BERT Embeddings (c1,c2)

In [ ]:
# from transformers import BertTokenizer, BertModel
# import torch

# # Memuat tokenizer dan model BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# # Fungsi untuk mendapatkan embedding dari teks menggunakan BERT
# def get_embedding(text):
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Mengambil embedding dari token [CLS] (index 0)
#     return outputs.last_hidden_state[:, 0, :].numpy()

# # Menghitung embedding untuk kolom c1 dan c2
# data['c1_embedding'] = data['c1'].apply(lambda x: get_embedding(x))
# data['c2_embedding'] = data['c2'].apply(lambda x: get_embedding(x))

BERT Embeddings (c1_context, c2_context)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Memuat tokenizer dan model BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Fungsi untuk mendapatkan embedding dari teks menggunakan BERT
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Mengambil embedding dari token [CLS] (index 0)
    return outputs.last_hidden_state[:, 0, :].numpy()

# Menghitung embedding untuk kolom c1 dan c2
data['c1_context_embedding'] = data['c1_context'].apply(lambda x: get_embedding(x))
data['c2_context_embedding'] = data['c2_context'].apply(lambda x: get_embedding(x))

In [16]:
# Mengubah embedding menjadi array NumPy
data['c1_context_embedding'] = data['c1_context_embedding'].apply(lambda x: np.array(x))
data['c2_context_embedding'] = data['c2_context_embedding'].apply(lambda x: np.array(x))

# Mengubah embedding menjadi array NumPy
data['c1_embedding'] = data['c1_embedding'].apply(lambda x: np.array(x))
data['c2_embedding'] = data['c2_embedding'].apply(lambda x: np.array(x))

In [17]:
data.to_json('../data/final_data/temp.jsonl', orient='records', lines=True)

In [4]:
data = pd.read_json('../data/final_data/temp.jsonl', lines=True)

Cosine Similarity

In [9]:
# Fungsi untuk menghitung cosine similarity
def cosine_similarity(c1_embedding, c2_embedding):
    c1_embedding = np.array(c1_embedding).flatten()
    c2_embedding = np.array(c2_embedding).flatten()
    return 1 - cosine(c1_embedding, c2_embedding)  # 1 - cosine distance

data['cosine_similarity'] = data.apply(lambda row: cosine_similarity(row['c1_embedding'], row['c2_embedding']), axis=1)

# Buat DataFrame baru
deep_learning_no_context = data[['c1', 'c2', 'c1_embedding', 'c2_embedding', 'cosine_similarity', 'match']]


In [ ]:
deep_learning_no_context

In [11]:
deep_learning_no_context.to_json('../data/final_data/deep_learning_no_context.jsonl', orient='records', lines=True)

Selain cosine similarity bisa tambahkan:
- L1 (Manhattan Distance)
- L2 (Euclidean Distance)
- L-infinity (Chebyshev Distance)


### Dengan Konteks
Coba dengan dan tanpa preprocessing context

Notes

Real Use Case: ketika ada 25000an entitas, dan dikasih 1 entitas, diminta untuk matching dia sebenernya entitas yang mana.

Untuk case fauzan: Setelah coba selesaikan binary classif, masuk ke real use case.

-Kendala:
Kalau pakai algo machine learning (XGBoost), gimana cara lookup data.

Ketika punya data baru, bandingin dengan cosine similarity (mana yg dekat)


Evaluations:
- Binary classification hitnung accuracy, dll
- Evaluasi look up

Notes

Model bert yang udah di pretrained, 

2500 entity (embedded) di store di numpy array untuk lookup. (metrics pake precission @1, @10 & recall)